In [1]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv("data/accepted_2007_to_2018Q4.csv",low_memory=False)

In [3]:
df.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,68407277,NaN,3600.0,3600.0,3600.0,36 months,13.99,123.03,C,C4,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,68355089,NaN,24700.0,24700.0,24700.0,36 months,11.99,820.28,C,C1,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
2,68341763,NaN,20000.0,20000.0,20000.0,60 months,10.78,432.66,B,B4,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
3,66310712,NaN,35000.0,35000.0,35000.0,60 months,14.85,829.90,C,C5,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
4,68476807,NaN,10400.0,10400.0,10400.0,60 months,22.45,289.91,F,F1,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df.shape

(2260701, 151)

In [12]:
# The Output value is the load_status
df['loan_status'].value_counts()

Fully Paid                                             1076751
Current                                                 878317
Charged Off                                             268559
Late (31-120 days)                                       21467
In Grace Period                                           8436
Late (16-30 days)                                         4349
Does not meet the credit policy. Status:Fully Paid        1988
Does not meet the credit policy. Status:Charged Off        761
Default                                                     40
Name: loan_status, dtype: int64

In [15]:
df = df[(df['loan_status']=='Fully Paid') | (df['loan_status']=='Charged Off')]

In [16]:
df.shape

(1345310, 151)

In [17]:
df.dtypes.value_counts()

float64    113
object      38
dtype: int64

In [11]:
check_nul = df.isnull().mean().sort_values()

In [12]:
check_nul

id                                            0.000000
fico_range_high                               0.000015
hardship_flag                                 0.000015
revol_bal                                     0.000015
initial_list_status                           0.000015
                                                ...   
hardship_reason                               0.995171
hardship_dpd                                  0.995171
hardship_loan_status                          0.995171
orig_projected_additional_accrued_interest    0.996173
member_id                                     1.000000
Length: 151, dtype: float64

In [14]:
# Delete any rows that have null values more than 30%
nul = check_nul[check_nul>0.3]

In [15]:
nul_col = nul.sort_values(ascending = False).index

In [16]:
data = df.drop(nul_col, axis = 1)

In [17]:
data.shape

(2260701, 93)

In [18]:
data.head()

,id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,...,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,disbursement_method,debt_settlement_flag
0,68407277,3600.0,3600.0,3600.0,36 months,13.99,123.03,C,C4,leadman,...,0.0,0.0,0.0,178050.0,7746.0,2400.0,13734.0,N,Cash,N
1,68355089,24700.0,24700.0,24700.0,36 months,11.99,820.28,C,C1,Engineer,...,7.7,0.0,0.0,314017.0,39475.0,79300.0,24667.0,N,Cash,N
2,68341763,20000.0,20000.0,20000.0,60 months,10.78,432.66,B,B4,truck driver,...,50.0,0.0,0.0,218418.0,18696.0,6200.0,14877.0,N,Cash,N
3,66310712,35000.0,35000.0,35000.0,60 months,14.85,829.90,C,C5,Information Systems Officer,...,0.0,0.0,0.0,381215.0,52226.0,62500.0,18000.0,N,Cash,N
4,68476807,10400.0,10400.0,10400.0,60 months,22.45,289.91,F,F1,Contract Specialist,...,60.0,0.0,0.0,439570.0,95768.0,20300.0,88097.0,N,Cash,N


KeyError: 'load_status'

In [23]:
# Check for categorical values
categorical = [feature for feature in data.columns if data[feature].dtype == "O"]
len(categorical)

24

In [24]:
for value in categorical:
    print(value)
    print(data[value].nunique())

id
2260701
term
2
grade
7
sub_grade
35
emp_title
512694
emp_length
11
home_ownership
6
verification_status
3
issue_d
139
loan_status
9
pymnt_plan
2
url
2260668
purpose
14
title
63155
zip_code
956
addr_state
51
earliest_cr_line
754
initial_list_status
2
last_pymnt_d
136
last_credit_pull_d
141
application_type
2
hardship_flag
2
disbursement_method
2
debt_settlement_flag
2


In [25]:
# There are multiple columns that have too many unique values, we need to drop them. Dropping any column that have more than
# 40 unique values
for value in categorical:
    if data[value].nunique()>40:
        data.drop([value],axis=1,inplace=True)

In [26]:
data.shape

(2260701, 83)

In [28]:
categorical = [feature for feature in data.columns if data[feature].dtype == "O"]
categorical

['term',
 'grade',
 'sub_grade',
 'emp_length',
 'home_ownership',
 'verification_status',
 'loan_status',
 'pymnt_plan',
 'purpose',
 'initial_list_status',
 'application_type',
 'hardship_flag',
 'disbursement_method',
 'debt_settlement_flag']